In [1]:
from rich import print
import os

from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware 
from langchain.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command

In [2]:
@tool('write_file')
def write_file_tool(file_path: str, content: str) -> str:
    """
    Write content to a file
    
    Args:
        file_path: The path to the file to write to
        content: The content to write to the file
    """

    return f"File written to {file_path} successfully"

@tool('execute_sql')
def execute_sql_tool(query: str) -> str:
    """
    Execute a SQL query
    
    Args:
        query: The SQL query to execute
    """
    
    return f"SQL {query} executed successfully"

@tool('read_data')
def read_data_tool(query: str, limit: int = 10) -> str:
    """
    Read data from a database
    
    Args:
        query: The SQL query to execute
        limit: The maximum number of rows to return
    """
    
    return f"{limit} rows of data read successfully by {query}"

In [3]:
agent = create_agent(
    model="gpt-4o",
    tools=[write_file_tool, execute_sql_tool, read_data_tool],
    middleware=[
        HumanInTheLoopMiddleware( 
            interrupt_on={
                "write_file": True,  # All decisions (approve, edit, reject) allowed
                "execute_sql": {"allowed_decisions": ["approve", "reject"]},  # No editing allowed
                # Safe operation, no approval needed
                "read_data": False,
            },
            # Prefix for interrupt messages - combined with tool name and args to form the full message
            # e.g., "Tool execution pending approval: execute_sql with query='DELETE FROM...'"
            # Individual tools can override this by specifying a "description" in their interrupt config
            description_prefix="Tool execution pending approval",
        ),
    ],
    # Human-in-the-loop requires checkpointing to handle interrupts.
    # In production, use a persistent checkpointer like AsyncPostgresSaver.
    checkpointer=InMemorySaver(),  
)

In [ ]:
from langgraph.types import Command

# Human-in-the-loop leverages LangGraph's persistence layer.
# You must provide a thread ID to associate the execution with a conversation thread,
# so the conversation can be paused and resumed (as is needed for human review).
config = {"configurable": {"thread_id": "some_id"}} 
# Run the graph until the interrupt is hit.
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "請刪除 postgres 中 consumer db 中 orders 資料表， created_at 早於 2024-01-01 的紀錄。",
            }
        ]
    },
    config=config 
)

# The interrupt contains the full HITL request with action_requests and review_configs
print(result['__interrupt__'])

KeyError: '__interrupt__'

In [5]:
print(result)

{
    'messages': [
        HumanMessage(
            content='Delete old records from the postgresdatabase',
            additional_kwargs={},
            response_metadata={},
            id='8c853ca8-63a7-4317-a387-5b3d2590811c'
        ),
        AIMessage(
            content='To delete old records from a PostgreSQL database, we need to determine which records are 
considered "old." This typically involves specifying a condition, such as older than a certain date in a date 
column.\n\nCould you please provide more details about the table from which you\'d like to delete records? 
Specifically, I need to know:\n- The name of the table.\n- The column that determines when a record is considered 
old (e.g., a date column).\n- The condition that makes a record "old" (e.g., older than a certain date).\n\nThis 
information will help me construct the correct SQL query to delete the records.',
            additional_kwargs={'refusal': None},
            response_metadata={
                'token_usage': {
                    'completion_tokens': 126,
                    'prompt_tokens': 164,
                    'total_tokens': 290,
                    'completion_tokens_details': {
                        'accepted_prediction_tokens': 0,
                        'audio_tokens': 0,
                        'reasoning_tokens': 0,
                        'rejected_prediction_tokens': 0
                    },
                    'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
                },
                'model_provider': 'openai',
                'model_name': 'gpt-4o-2024-08-06',
                'system_fingerprint': 'fp_a4d13246c5',
                'id': 'chatcmpl-CmsRftWXtGHvW5IbKGGsFVA5Kr81u',
                'service_tier': 'default',
                'finish_reason': 'stop',
                'logprobs': None
            },
            id='lc_run--019b1fcc-b278-7463-bc3f-59da392a4d22-0',
            usage_metadata={
                'input_tokens': 164,
                'output_tokens': 126,
                'total_tokens': 290,
                'input_token_details': {'audio': 0, 'cache_read': 0},
                'output_token_details': {'audio': 0, 'reasoning': 0}
            }
        )
    ]
}

In [ ]:
# Resume with approval decision
agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}  # or "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)